In [100]:
import seqdata as sd
import xarray as xr
import numpy as np
import dask.array as da
import dask_ml as dml
from graph_part import train_test_validation_split

In [4]:
sdata = sd.open_zarr('/cellar/users/dlaub/projects/ML4GLand/use_cases/kopp21/kopp21.zarr')
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 1013080, cov_sample: 1, _length: 500)
Coordinates:
  * cov_sample  (cov_sample) object 'stam'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(63318,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(63318,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(63318,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(4096, 1, 500), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(4096, 500), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(63318,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

In [43]:
def train_test_split_chrom(sdata: xr.Dataset, test_chroms: list[str], train_key=None):
    test_mask = sdata.chrom.isin(test_chroms).compute()
    return sdata.assign()
    return sdata.sel(_sequence=~test_mask), sdata.sel(_sequence=test_mask)

In [45]:
splits = train_test_split_chrom(sdata, ['chr2', 'chr3'])
[s.sizes['_sequence'] for s in splits]

[875547, 137533]

In [62]:
def train_test_split_random(sdata: xr.Dataset, dim: str, groups=None, test_size=0.1, random_state=None):
    splitter = dml.model_selection.ShuffleSplit(
        n_splits=1,
        test_size=test_size,
        random_state=random_state
    )
    train_idx, test_idx = next(splitter.split(da.arange(sdata.sizes[dim]), groups=groups))
    return sdata.isel({dim: train_idx}), sdata.isel({dim: test_idx})

In [71]:
splits = train_test_split_random(sdata, '_sequence', sdata.chrom)
[s.sizes['_sequence'] for s in splits]

[911772, 101308]

In [105]:
def train_test_split_homology(sdata: xr.Dataset, seq_var: str, test_size=0.1, nucleotide=True):
    seq_length = sdata.sizes[sdata.attrs['length_dim']]
    outs = train_test_validation_split(
        sdata[seq_var].to_numpy().view(f'S{seq_length}').squeeze().astype('U').astype(object),
        test_size=test_size,
        initialization_mode='fast-nn',
        nucleotide=nucleotide,
        prefilter=True,
        denominator='shortest'
    )
    train_idx, test_idx = map(np.array, outs)
    return sdata.isel({sdata.attrs['sequence_dim']: train_idx}), sdata.isel({sdata.attrs['sequence_dim']: test_idx})

Splitting by homology with 10,000 sequences takes ~6 minutes.

In [88]:
train, test = train_test_split_homology(sdata.isel(_sequence=slice(0, int(1e4))), 'seq')

Unable to join threads to shut down before fork(). This can break multithreading in child processes.

temp/seq_db exists and will be overwritten
createdb --dbtype 2 graphpart_api.fasta.tmp temp/seq_db 

MMseqs Version:       	14.7e284
Database type         	2
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[
Time for merging to seq_db_h: 0h 0m 0s 99ms
Time for merging to seq_db: 0h 0m 0s 117ms
Database type: Nucleotide
Time for processing: 0h 0m 0s 612ms
Unable to join threads to shut down before fork(). This can break multithreading in child processes.

prefilter -s 7.5 temp/seq_db temp/seq_db temp/pref 

MMseqs Version:           	14.7e284
Substitution matrix       	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix  	aa:VTML80.out,nucl:nucleotide.out
Sensitivity               	7.5
k-mer length              	0
k-score                   	seq:21474836

0it [00:00, ?it/s]

Full graph nr. of edges: 488825
1000 [1000]
Initialization mode fast-nn
1
Currently have this many samples: 10000
!  seq_0 seq_7030 {'metric': 0.544}  !
Need to remove! Currently have this many samples: 10000
After removal we have this many samples: 9867


Note: we don't demonstrate this for more than 100,000 sequences since this takes ~3.5 hours to run. The homology graph partitioning algorithm needs approximately $O(n^2)$ time so increasing the amount of sequences beyond this would be intractable for the purposes of a tutorial.

In [106]:
train, test = train_test_split_homology(sdata.isel(_sequence=slice(0, int(1e5))), 'seq')

Unable to join threads to shut down before fork(). This can break multithreading in child processes.

createdb --dbtype 2 graphpart_api.fasta.tmp temp/seq_db 

MMseqs Version:       	14.7e284
Database type         	2
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=========
Time for merging to seq_db_h: 0h 0m 0s 182ms
Time for merging to seq_db: 0h 0m 0s 345ms
Database type: Nucleotide
Time for processing: 0h 0m 2s 966ms
Unable to join threads to shut down before fork(). This can break multithreading in child processes.

prefilter -s 7.5 temp/seq_db temp/seq_db temp/pref 

MMseqs Version:           	14.7e284
Substitution matrix       	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix  	aa:VTML80.out,nucl:nucleotide.out
Sensitivity               	7.5
k-mer length              	0
k-score                   	seq:2147483647,prof:2147483647
Alphabet size 

0it [00:00, ?it/s]

Full graph nr. of edges: 4977945
10000 [10000]
Initialization mode fast-nn
edges: 10000
{'cluster': 3272, 'C-size': 8085, 'label-counts': array([8085])} {'metric': 0.4}
edges: 20000
{'cluster': 14429, 'C-size': 75, 'label-counts': array([75])} {'metric': 0.4}
edges: 30000
{'cluster': 25934, 'C-size': 11, 'label-counts': array([11])} {'metric': 0.4}
edges: 40000
{'cluster': 36944, 'C-size': 94, 'label-counts': array([94])} {'metric': 0.4}
edges: 50000
{'cluster': 47984, 'C-size': 10, 'label-counts': array([10])} {'metric': 0.4}
edges: 60000
{'cluster': 59133, 'C-size': 8, 'label-counts': array([8])} {'metric': 0.4}
edges: 70000
{'cluster': 70268, 'C-size': 57, 'label-counts': array([57])} {'metric': 0.4}
edges: 80000
{'cluster': 81346, 'C-size': 31, 'label-counts': array([31])} {'metric': 0.4}
edges: 90000
{'cluster': 92025, 'C-size': 22, 'label-counts': array([22])} {'metric': 0.4}
1
Currently have this many samples: 100000
!  seq_0 seq_19522 {'metric': 0.5920000000000001}  !
Need to r